In [2]:
import os

# 假设有两个文件路径
file1_path = '/home/disk3/jsa/projects/TLD_backend/src/main/algorithm/main/main.py'
file2_path = '/home/disk3/jsa/projects/TLD_backend/src/main/java/org/zjuvipa/util/PythonCallerUtil.java'
file3_path = '/home/disk3/jsa/projects/TLD_backend/src/main/java/org/zjuvipa/DemoApplication.java'
# 计算相对路径
relative_path = os.path.relpath(file1_path, os.path.dirname(file3_path))

print("Relative path from file2 to file1:", relative_path)

Relative path from file2 to file1: ../../../algorithm/main/main.py
